In [1]:
import os
import cv2
import numpy as np

import torch
from torch.utils.data import DataLoader

from data.yolo_dataset import YoloDataset, collate_fn

In [ ]:
from torch import nn


class YOLOv3Loss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()                       ## Do I have to set reduction? 
        self.multiMargin = nn.MultiLabelSoftMarginLoss()        ## Do I have to set reduction? (2)
                                                                ## https://cvml.tistory.com/26

        
    def forward(self, pred, target, scale, anchors):

        ## no_obj_loss(No Object Loss):     Loss for objectness score      of non-object-assigned BBOXes
        ## is_obj_loss(Object Loss):        Loss for objectness score      of     object-assigned BBOXes
        ## coord_loss(Coordinates Loss):    Loss for predicted coordinates of     object-assigned BBOXes
        ## class_loss(Classification Loss): Loss for predicted class-ids   of     object-assigned BBOXes 

        is_assigned = pred[..., 4] == 1     ## tensor([(element == 1) for element in 4th column of pred])   ## e.g. tensor([True, False, False, ...])
        no_assigned = pred[..., 4] == 0     ## If use these boolean-list tensor as a indices,
                                            ##    we can extract the only rows from target(label) tensor -- whose 4th column element(objectness score) is 1-or-0

        scale = torch.tensor(scale).reshape(-1, 1).repeat(1, 2).reshape(1, 1, 1, 1, 2)      ## 13 -> [[[[[13, 13]]]]]
        cell_offset = (pred[..., :2].floor_divide(scale)) * scale                           ## ??????????????????
        pred[..., :2] = pred[..., :2] - cell_offset

        no_obj_loss = self.get_loss(pred[...,  4][no_assigned], target[...,  4][no_assigned], anchors, opt="NO_OBJ")
        is_obj_loss = self.get_loss(pred[...,  4][is_assigned], target[...,  4][is_assigned], anchors, opt="IS_OBJ")
        coord_loss =  self.get_loss(pred[..., :4][is_assigned], target[..., :4][is_assigned], anchors, opt="COORD")
        class_loss =  self.get_loss(pred[..., 5:][is_assigned], target[..., 5:][is_assigned], anchors, opt="CLASS")
        
        loss = no_obj_loss + is_obj_loss + coord_loss + class_loss
        return loss


    def get_loss(self, pred, target, anchors, opt):
        
        if opt == "NO_OBJ":
            loss = self.bce(pred, target)
            return loss

        elif opt == "IS_OBJ":
            loss = self.bce(torch.sigmoid(pred), target)            ## If use [wh_IOU * target] instead of [target], MSE loss is better . . . maybe.
            return loss                                             ##    cause [target] and [wh_IOU * target] values differ in "Discrete"/"Continuous"

        elif opt == "COORD":
            pred_bboxes =   torch.cat([torch.sigmoid(pred[..., 0:2]), torch.exp(pred[..., 2:4])          ], dim=1)
            target_bboxes = torch.cat([              pred[..., 0:2] ,          (pred[..., 2:4] / anchors)], dim=1)
            loss = self.mse(pred_bboxes, target_bboxes)
            return loss

        elif opt == "CLASS":
            loss = self.multiMargin(pred, target)
            return loss

In [2]:
def train(
        model,
        train_loader,
        loss_func,
        dataset_option,
        model_option,
        epoch,
        # anchors,
        ):
    model.train()

    scales = torch.tensor(model_option["YOLOv3"]["SCALES"]).to(device='cpu')       ## [13, 26, 52]
    anchors = torch.tensor(model_option["YOLOv3"]["ANCHORS"]).to(device='cpu')

    for i, batch_img, batch_label, batch_img_path in enumerate(train_loader, 0):
        batch_size = batch_img.size(0)
        
        #################
        ##  FORWARDING ##
        #################
        pred = model(batch_img)                                                      ### batch_img: tensor(   N, 3, 416, 416) . . . . . . . . . . . N = batch_size
        loss = ( loss_func(pred[0], batch_label[0], scales[0], anchor=anchors[0])    ######## pred: tensor(3, N, 3, S, S, 1 + 4 + class_offset) . . S = scale_size
               + loss_func(pred[1], batch_label[1], scales[1], anchor=anchors[1])    # batch_label: tensor(3, N, 3, S, S, 1 + 4 + class_offset)
               + loss_func(pred[2], batch_label[2], scales[2], anchor=anchors[2]) )  ##### anchors: tensor(3,    3,       2) . . . is list of pairs(anch_w, anch_h)

        #################
        ## BACKWARDING ##
        #################
        loss.backward()
    

In [ ]:
def valid(
    model,
    valid_loader,
    model_option,
    epoch,
    # anchors,
    ):
    model.eval()
    true_pred_num = 0
    gt_num = 0

    for i, batch_img, batch_label, batch_img_path in enumerate(valid_loader, 0):

        pred = model(batch_img)

        ## Post-Processing?

        ## Get the number of both true predictions and ground truth


    ## Examine Accuracy
    acc = (true_pred_num / gt_num + 1e-16) * 100
    
    return acc

In [6]:
dataset_option = {  "DATASET": {
                        "NAME": "ship",
                        "ROOT": "../datasets/ship",
                        "CLASSES": {
                            #    "선박": 0, "부표": 1, "어망부표": 2,
                            #    "해상풍력": 3, "등대": 4, "기타부유물" : 5
                               "선박": 0, "부표": 1, "어망부표": 1,
                               "해상풍력": 1, "등대": 1, "기타부유물" : 1
                        },
                        "NUM_CLASSES": 2
                     }
                 }

model_option = {"YOLOv3": {
                     "SCALES": [13, 26, 52],
                     "NUM_ANCHORS": 9,
                     "ANCHORS": [[( 10, 13), ( 16,  30), ( 33,  23)],
                                 [( 30, 61), ( 62,  45), ( 59, 119)],
                                 [(116, 90), (156, 198), (373, 326)]]
                    }
               }

optim_option = {"OPTIMIZER": {
                     "METHOD": "adam",
                     "BATCH_SIZE": 32,
                     "EPOCHS": 10,
                     "LR": 1e-4,
                    }
               }

In [7]:
epochs = optim_option["OPTIMIZER"]["EPOCHS"]
batch_size = optim_option["OPTIMIZER"]["BATCH_SIZE"]

In [ ]:
model = Net()
loss_function = YOLOv3Loss()

In [ ]:
##############
## DATALOAD ##
##############
train_dataset = YoloDataset(dataset_option, model_option, split="train")
train_loader = DataLoader(train_dataset, batch_size, collate_fn=collate_fn)
valid_dataset = YoloDataset(dataset_option, model_option, split="valid")
valid_loader = DataLoader(valid_dataset, batch_size, collate_fn=collate_fn)

In [ ]:
for epoch in range(epochs):
    ###########
    ## TRAIN ##
    ###########
    train(  
            model,
            train_loader,
            loss_function,
            dataset_option,
            model_option,
            epoch,
            # anchors,
          )
        
    #######################
    ## VALID (INFERENCE) ##
    #######################
    acc = valid(
                 model,
                 valid_loader,
                 model_option,
                 epoch,
                 # anchors,
               )

    print(f"Epoch: ({epoch + 1}/{epochs}) . . . [acc: {acc:.2f}]")
    